In [19]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv
import json

In [20]:
#define state schema
class BatsmanState(TypedDict):
    runs: int
    balls: int
    fours: int
    sixes: int

    sr: float
    bpb: float
    boundary_percentage: float

In [21]:
#load environment variables from a .env file
load_dotenv()

#initiate the language model
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
#model = ChatOpenAI(model_name="gpt-4", temperature=0)

In [22]:
#calculate strike rate
def calculate_sr(state: BatsmanState):
   sr =  (state['runs'] / state['balls']) * 100
   sr = round(sr, 2)
   return {'sr': sr}

In [23]:
#calculate balls per boundary
def calculate_bpb(state: BatsmanState):
   bpb = state['balls'] / (state['fours'] + state['sixes'])
   bpb = round(bpb, 2)
   return {'bpb': bpb}

In [24]:
#calculate boundary percentage
def calculate_boundary_percentage(state: BatsmanState):
   boundary_percentage = ((state['fours'] * 4 + state['sixes'] * 6) / state['runs']) * 100
   boundary_percentage = round(boundary_percentage, 2)
   return {'boundary_percentage': boundary_percentage}

In [25]:
#calculate summary
def summary(state: BatsmanState):
    summary = f"""
    Batsman Performance Summary:
    Strike Rate: {state['sr']}
    BPB: {state['bpb']}
    Boundary Percentage: {state['boundary_percentage']}
    """

   
    return {'summary': summary}

In [26]:
#define the node functions
graph = StateGraph(BatsmanState)
graph.add_node('calculate_sr', calculate_sr)
graph.add_node('calculate_bpb', calculate_bpb)
graph.add_node('calculate_boundary_percentage', calculate_boundary_percentage)
graph.add_node('summary', summary)

#define the edges
graph.add_edge(START, 'calculate_sr')
graph.add_edge(START, 'calculate_bpb')
graph.add_edge(START, 'calculate_boundary_percentage')
graph.add_edge('calculate_sr', 'summary')
graph.add_edge('calculate_bpb', 'summary')
graph.add_edge('calculate_boundary_percentage', 'summary')


graph.add_edge('summary', END)

workflow = graph.compile()




In [27]:
initial_state = {
    'runs': 180,
    'balls':80,
    'fours': 10,
    'sixes': 5,
}

final_state = workflow.invoke(initial_state)
print(final_state)

{'runs': 180, 'balls': 80, 'fours': 10, 'sixes': 5, 'sr': 225.0, 'bpb': 5.33, 'boundary_percentage': 38.89}
